<a href="https://colab.research.google.com/github/Lossophy/BINA-Projekt/blob/main/import%20git%20-%20Kopie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q xlrd
!git clone https://github.com/Lossophy/BINA-Projekt.git

Cloning into 'BINA-Projekt'...
remote: Enumerating objects: 86, done.
remote: Counting objects: 100% (86/86), done.
remote: Compressing objects: 100% (72/72), done.
remote: Total 86 (delta 43), reused 33 (delta 12), pack-reused 0 (from 0)
Receiving objects: 100% (86/86), 741.43 KiB | 3.12 MiB/s, done.
Resolving deltas: 100% (43/43), done.


In [27]:
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
import plotly.express as px
from IPython.display import display
from IPython.display import clear_output
import plotly.io as pio
import plotly.graph_objects as go
pio.renderers.default = 'notebook'  # oder 'iframe' oder 'plotly_mimetype'


In [48]:
df = pd.read_csv('/content/BINA-Projekt/Data/ogd115_gest_bilanz.csv')

In [4]:
df.head()

,Jahr,Rubrik,Energietraeger,TJ
0,1980,Bruttoverbrauch,Elektrizität,-29450.0
1,1980,Bruttoverbrauch,Erdölprodukte,321250.0
2,1980,Bruttoverbrauch,Fernwärme,0.0
3,1980,Bruttoverbrauch,Gas,36280.0
4,1980,Bruttoverbrauch,Holzenergie,26280.0


In [43]:
von_jahr_widget = widgets.Dropdown(
    options=sorted(df['Jahr'].unique()),
    description='Jahr',
    disabled=False,
)

In [44]:
bis_jahr_widget = widgets.Dropdown(
    options=sorted(df['Jahr'].unique()),
    description='Jahr',
    disabled=False,
)

In [45]:
rubrik_widget = widgets.Dropdown(
    options=sorted(df['Rubrik'].unique()),
    value=df['Rubrik'].unique()[0],
    description='Rubrik:'
)


In [49]:
energietraeger_widget = widgets.Dropdown(
    options=sorted(df['Energietraeger'].unique()),
    value=df['Energietraeger'].unique()[0],
    description='energietraeger:'
)


In [50]:
multi_energietraeger_widget = widgets.SelectMultiple(
    options=sorted(df['Energietraeger'].unique()),
    value=tuple(df['Energietraeger'].unique()),
    description='Energie:'
)

In [51]:
# Interaktive Funktion definieren
def plot_daten(jahr, rubrik, energietraeger):
    gefiltert = df[(df['Jahr'] == jahr) &
                   (df['Rubrik'] == rubrik) &
                   (df['Energietraeger'] == energietraeger)]

    if gefiltert.empty:
        print("Keine Daten für diese Kombination.")
        return

    plt.figure(figsize=(6,4))
    plt.bar([0], gefiltert['TJ'], tick_label=[f"{jahr}"])
    plt.ylabel('TJ')
    plt.title(f"{rubrik} - {energietraeger} ({jahr})")
    plt.show()

In [52]:
# Interaktive Ansicht verbinden
ui = widgets.VBox([von_jahr_widget, rubrik_widget, energietraeger_widget])
out = widgets.interactive_output(plot_daten, {
    'jahr': von_jahr_widget,
    'rubrik': rubrik_widget,
    'energietraeger': energietraeger_widget
})

display(ui, out)

Output()

In [39]:
# Plot-Funktion
def plot_daten(von_jahr, bis_jahr, rubrik, energietraeger):
    if von_jahr > bis_jahr:
        print("Fehler: 'Von Jahr' darf nicht größer als 'Bis Jahr' sein.")
        return

    gefiltert = df[(df['Jahr'] >= von_jahr) &
                   (df['Jahr'] <= bis_jahr) &
                   (df['Rubrik'] == rubrik) &
                   (df['Energietraeger'] == energietraeger)]

    if gefiltert.empty:
        print("Keine Daten für diese Auswahl.")
        return

    gruppiert = gefiltert.groupby('Jahr')['TJ'].sum().reset_index()

    plt.figure(figsize=(25,10))
    plt.plot(gruppiert['Jahr'].astype(int), gruppiert['TJ'], marker='o')
    plt.xticks(gruppiert['Jahr'].astype(int))
    plt.xlabel('Jahr')
    plt.ylabel('TJ')
    plt.title(f'{rubrik} - {energietraeger} ({von_jahr}–{bis_jahr})')
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# Anzeige
ui = widgets.VBox([von_jahr_widget, bis_jahr_widget, rubrik_widget, energietraeger_widget])
out = widgets.interactive_output(plot_daten, {
    'von_jahr': von_jahr_widget,
    'bis_jahr': bis_jahr_widget,
    'rubrik': rubrik_widget,
    'energietraeger': energietraeger_widget
})
clear_output()
display(ui, out)


Output()

In [37]:
# Plotfunktion
def pie_plot(von_jahr, rubrik):
    gefiltert = df[
        (df['Jahr'] == von_jahr) &
        (df['Rubrik'] == rubrik)
    ]

    if gefiltert.empty:
        print("⚠️ Keine Daten verfügbar für diese Auswahl.")
        return

    grouped = gefiltert.groupby('Energieträger')['TJ'].sum().reset_index()

    fig = go.Figure(data=[go.Pie(
        labels=grouped['Energieträger'],
        values=grouped['TJ'],
        hoverinfo='label+percent',
        textinfo='label+value'
    )])

    fig.update_layout(
        title=f'Verteilung der Energieträger – {rubrik} ({von_jahr})',
        height=500
    )

    fig.show()



In [38]:
# Interaktive Verbindung
ui = widgets.HBox([von_jahr_widget, rubrik_widget])
out = widgets.interactive_output(pie_plot, {
    'von_jahr': von_jahr_widget,
    'rubrik': rubrik_widget
})

display(ui, out)

Output()

In [19]:
# Alle Energieträger als Default setzen
alle_traeger = sorted(df['Energietraeger'].unique())
energietraeger_widget = widgets.SelectMultiple(
    options=alle_traeger,
    value=tuple(alle_traeger),  # Muss ein Tupel sein!
    description='Energie:'
)

In [24]:
# Plotfunktion
def area_plot(von_jahr, bis_jahr, rubrik, Energietraeger):
    if von_jahr > bis_jahr:
        print("⚠️ Fehler: Von-Jahr darf nicht größer als Bis-Jahr sein.")
        return

    if not Energietraeger:
        print("⚠️ Bitte mindestens einen Energieträger auswählen.")
        return

    gefiltert = df[
        (df['Jahr'] >= von_jahr) &
        (df['Jahr'] <= bis_jahr) &
        (df['Rubrik'] == rubrik) &
        (df['Energietraeger'].isin(list(Energietraeger)))
    ]

    if gefiltert.empty:
        print("⚠️ Keine Daten für diese Auswahl.")
        return

    grouped = gefiltert.groupby(['Jahr', 'Energietraeger'])['TJ'].sum().reset_index()

    fig = px.area(grouped, x="Jahr", y="TJ", color="Energietraeger",
                  title=f"TJ nach Energietraeger ({rubrik}, {von_jahr}–{bis_jahr})",
                  markers=True)

    fig.update_layout(
        yaxis_title='TJ',
        xaxis=dict(dtick=1),
        legend_title='Energietraeger'
    )
    fig.show()

In [26]:
# Interaktive Verbindung
ui = widgets.VBox([von_jahr_widget, bis_jahr_widget, rubrik_widget, energietraeger_widget])
out = widgets.interactive_output(area_plot, {
    'von_jahr': von_jahr_widget,
    'bis_jahr': bis_jahr_widget,
    'rubrik': rubrik_widget,
    'Energietraeger': energietraeger_widget
})

display(ui,out)

Output()